In [4]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

100% 813M/814M [00:08<00:00, 105MB/s] 
100% 814M/814M [00:08<00:00, 104MB/s]


In [5]:
!unzip -q dogs-vs-cats-redux-kernels-edition.zip -d .

In [6]:
!unzip -q train.zip -d .

In [9]:
%rm -rf dataset

In [10]:
import os
import tensorflow as tf

print( len(os.listdir('/content/train/')))

os.mkdir('/content/dataset')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')

# tf.keras 이용해서 이미지 숫자화
# tf.keras.proprocessing.image_dataset_from_directory()

25000


In [15]:
from re import I
import shutil
# 모든 고양이 사진은 cat폴더, 모든 개 사진은 dog 폴더

for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile('/content/train/'+i, '/content/dataset/cat/' + i)
  if 'dog' in i:
    shutil.copyfile('/content/train/'+i, '/content/dataset/dog/' + i)

In [25]:
# tf.keras 이용해서 이미지 숫자화
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size = (64,64),
    batch_size=32,              # 이미지 2만장 한번에 넣지 않고 batch 숫자만큼
    subset = 'training',        # 여기는 training set
    validation_split = 0.2,
    seed = 1234
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size = (64,64),
    batch_size=64,              # 이미지 2만장 한번에 넣지 않고 batch 숫자만큼
    subset = 'validation',      # 여기는 validation set
    validation_split = 0.2,
    seed = 1234
)

print(train_ds)

def 전처리함수(i, 정답):
  i = tf.cast(i/255.0, tf.float32)
  return i, 정답

train_ds = train_ds.map(전처리함수)
val_ds = val_ds.map(전처리함수)


#import matplotlib.pyplot as plt

#for i, 정답 in train_ds.take(1):
#  print(i)
#  print(정답)
#  plt.imshow( i[0].numpy().astype('uint8'))
#  plt.show()

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [30]:
model = tf.keras.Sequential([

    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal', input_shape=(64,64,3) ),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),


    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2) ),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2) ),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2) ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation= 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1,activation = "sigmoid"),
])
model.summary()

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.fit(train_ds, validation_data = val_ds, epochs=5)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_5 (RandomFlip)  (None, 64, 64, 3)         0         
                                                                 
 random_rotation_5 (RandomR  (None, 64, 64, 3)         0         
 otation)                                                        
                                                                 
 random_zoom_5 (RandomZoom)  (None, 64, 64, 3)         0         
                                                                 
 conv2d_21 (Conv2D)          (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 32, 32, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_22 (Conv2D)          (None, 32, 32, 64)       

In [31]:
model.save('새폴더/model1')

In [33]:
불러온모델 = tf.keras.models.load_model('새폴더/model1')
불러온모델.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_5 (RandomFlip)  (None, 64, 64, 3)         0         
                                                                 
 random_rotation_5 (RandomR  (None, 64, 64, 3)         0         
 otation)                                                        
                                                                 
 random_zoom_5 (RandomZoom)  (None, 64, 64, 3)         0         
                                                                 
 conv2d_21 (Conv2D)          (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 32, 32, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_22 (Conv2D)          (None, 32, 32, 64)       